# Rolling apply and mapping functions

https://youtu.be/uLqmM6ExPvo?list=PLQVvvaa0QuDc-3szzjeP6N6b0aDrrKyL-

In [9]:
import quandl;
import pandas as pd;

import pickle;

import matplotlib.pyplot as plt;
from matplotlib import style;
style.use("ggplot");

import numpy as np;

In [10]:
api_key = open("quandlapikey.txt", "r").read();

def mortgage_30y_resampled():
    df = quandl.get("FMAC/MORTG", trim_start = "1975-01-01", authtoken = api_key);       
    df["Value"] = (df["Value"] - df["Value"][0]) / df["Value"][0] * 100.0;
    df.columns = ["M30"];
    return df.resample("M").mean();

def state_list():
    fiddy_states = pd.read_html("https://simple.wikipedia.org/wiki/List_of_U.S._states");
    return fiddy_states[0][0][1:];

def grap_initial_state_data_start_pct():
    states = state_list();
    main_df = pd.DataFrame();
    for ab in states:
        querry = "FMAC/HPI_" + ab;
        df = quandl.get(querry, authtoken = api_key);
        df.columns = [ab];        
        df[ab] = (df[ab] - df[ab][0]) / df[ab][0] * 100.0; # <-------
        if main_df.empty:
            main_df = df;
        else:
            main_df = main_df.join(df);

    pickle_out = open("./data/fiddy_states.pickle", "wb");
    pickle.dump(main_df, pickle_out);
    pickle_out.close();
    
def HPI_Benchmark():
    df = quandl.get("FMAC/HPI_USA", authtoken = api_key);
    df.columns = ["US"];        
    df["US"] = (df["US"] - df["US"][0]) / df["US"][0] * 100.0; # <-------
    return df;


In [11]:
def sp500_data():
    df = quandl.get("YAHOO/INDEX_GSPC", trim_start = "1975-01-01", authtoken = api_key);
    df["Adjusted Close"] = (df["Adjusted Close"] - df["Adjusted Close"][0]) / df["Adjusted Close"][0] * 100.0; # <-------
    df = df.resample("M").mean();
    df.rename(columns={"Adjusted Close":"sp500"}, inplace = True);
    df = df["sp500"];
    return df;

In [12]:
df = sp500_data();
print(df.head());

Date
1975-01-31     3.323491
1975-02-28    14.049322
1975-03-31    19.367785
1975-04-30    20.636734
1975-05-31    28.287322
Freq: M, Name: sp500, dtype: float64


In [13]:
def gdp_data():
    df = quandl.get("BCB/4385", trim_start = "1975-01-01", authtoken = api_key);
    df["Value"] = (df["Value"] - df["Value"][0]) / df["Value"][0] * 100.0; # <-------
    df = df.resample("M").mean();
    df.rename(columns={"Value":"GDP"}, inplace = True);
    df = df["GDP"];
    return df;
    

In [14]:
def us_unemployment():
    df = quandl.get("ECPI/JOB_G", trim_start = "1975-01-01", authtoken = api_key);
    df["Unemployment Rate"] = (df["Unemployment Rate"] - df["Unemployment Rate"][0]) / df["Unemployment Rate"][0] * 100.0; # <-------
    df = df.resample("1D").mean();
    df = df.resample("M").mean();
    return df;

In [15]:
sp500 = sp500_data();
US_GDP = gdp_data();
US_uneployment = us_unemployment();

m30 = mortgage_30y_resampled();
HPI_data = pd.read_pickle("./data/fiddy_states.pickle");
HPI_bench = HPI_Benchmark();

In [16]:
HPI = HPI_data.join([HPI_bench, m30, US_uneployment, US_GDP, sp500]);
print(HPI);
print(HPI.corr());
# we have nans!!

                    AL          AK          AZ          AR           CA  \
Date                                                                      
1975-01-31    0.000000    0.000000    0.000000    0.000000     0.000000   
1975-02-28    0.626500    1.485775    1.688504    0.846192     0.356177   
1975-03-31    1.358575    3.006473    3.261346    1.581956     1.575690   
1975-04-30    2.254726    4.593530    4.475810    2.183669     3.573196   
1975-05-31    3.107829    6.327600    5.139617    2.786248     5.241395   
1975-06-30    3.585231    8.184557    5.035190    3.444136     5.987360   
1975-07-31    3.502089   10.041682    4.062221    4.024416     6.773494   
1975-08-31    2.956561   11.733579    2.489967    4.384871     8.176235   
1975-09-30    2.287053   13.207406    0.830264    4.436568     9.724202   
1975-10-31    1.937454   14.547916   -0.101833    4.351232    11.144933   
1975-11-30    2.169245   15.744312    0.285140    4.481393    12.103889   
1975-12-31    3.058008   

In [17]:
HPI.dropna(inplace = True);
print(HPI);
print(HPI.corr());

                    AL          AK          AZ          AR          CA  \
Date                                                                     
1990-01-31   97.377466   62.960664  125.925384   90.013079  407.639219   
1990-02-28   97.306825   64.745987  125.954481   90.292939  409.393746   
1990-03-31   97.640926   68.843816  125.677184   90.749096  412.358848   
1990-04-30   98.347335   75.545290  125.537118   91.190719  414.727667   
1990-05-31   99.135672   84.269489  125.820526   91.693828  415.785301   
1990-06-30   99.593283   93.030215  126.319903   92.282205  417.279958   
1990-07-31   99.601506   99.468341  126.555572   92.875334  419.261081   
1990-08-31   99.558429  102.479213  126.226122   93.524309  419.631783   
1990-09-30   99.776456  102.769503  125.371627   94.010613  417.928706   
1990-10-31  100.047438  101.770286  124.149933   94.146892  414.911709   
1990-11-30   99.956110  100.506280  123.078108   94.158401  411.244025   
1990-12-31   99.546794   99.752104  12

In [18]:
HPI.to_pickle("./data/HPI.pickle");

in fact, this part starts here

In [19]:
housing_data = pd.read_pickle("./data/HPI.pickle");
housing_data = housing_data.pct_change();
print(housing_data.head());

                  AL        AK        AZ        AR        CA        CO  \
Date                                                                     
1990-01-31       NaN       NaN       NaN       NaN       NaN       NaN   
1990-02-28 -0.000725  0.028356  0.000231  0.003109  0.004304  0.001633   
1990-03-31  0.003433  0.063291 -0.002202  0.005052  0.007243  0.001544   
1990-04-30  0.007235  0.097343 -0.001114  0.004866  0.005745  0.000830   
1990-05-31  0.008016  0.115483  0.002258  0.005517  0.002550  0.005127   

                  CT        DE        FL        GA    ...           VA  \
Date                                                  ...                
1990-01-31       NaN       NaN       NaN       NaN    ...          NaN   
1990-02-28 -0.008158  0.006780 -0.002739 -0.004290    ...    -0.001692   
1990-03-31 -0.009803  0.001564 -0.000640 -0.006709    ...     0.001023   
1990-04-30 -0.011515 -0.000451  0.002636 -0.001817    ...     0.005560   
1990-05-31 -0.011205  0.000009  0.003

In [20]:
housing_data.replace([np.inf, -np.inf], np.nan, inplace = True);
housing_data.dropna(inplace = True);
print(housing_data.head());

                  AL        AK        AZ        AR        CA        CO  \
Date                                                                     
1990-03-31  0.003433  0.063291 -0.002202  0.005052  0.007243  0.001544   
1990-04-30  0.007235  0.097343 -0.001114  0.004866  0.005745  0.000830   
1990-05-31  0.008016  0.115483  0.002258  0.005517  0.002550  0.005127   
1990-06-30  0.004616  0.103961  0.003969  0.006417  0.003595  0.007326   
1990-07-31  0.000083  0.069205  0.001866  0.006427  0.004748  0.003203   

                  CT        DE        FL        GA    ...           VA  \
Date                                                  ...                
1990-03-31 -0.009803  0.001564 -0.000640 -0.006709    ...     0.001023   
1990-04-30 -0.011515 -0.000451  0.002636 -0.001817    ...     0.005560   
1990-05-31 -0.011205  0.000009  0.003710  0.003266    ...     0.005750   
1990-06-30 -0.007019 -0.000411  0.003081  0.002858    ...     0.003002   
1990-07-31 -0.003224 -0.003613  0.002

In [22]:
housing_data["US_HPI_future"] = housing_data["US"].shift(-1);
print(housing_data[["US_HPI_future", "US"]].head());

            US_HPI_future        US
Date                               
1990-03-31       0.005182  0.004323
1990-04-30       0.005244  0.005182
1990-05-31       0.005103  0.005244
1990-06-30       0.003712  0.005103
1990-07-31       0.000489  0.003712


In [ ]:
def cre